In [1]:
import pandas as pd

municipios = pd.read_excel('resources/elecciones.xlsx', index_col=False)

In [60]:
# Arreglor el Nombre de las columnas de la tabla para poder acceder a ellas usando el "."
municipios.columns = [i.strip().replace(' ','_') for i in municipios.columns]

fix_col_names = list(map(lambda nombre: nombre.replace(' ','_'), col_names))
municipios.columns = fix_col_names

# Arreglar el los espacios adicionales en los valores string de la tabla (columnas 0, 2 y 4)
municipios.Nombre_de_Comunidad = municipios.Nombre_de_Comunidad.apply(lambda i: i.strip())
municipios.Nombre_de_Provincia = municipios.Nombre_de_Provincia.apply(lambda i: i.strip())
municipios.Nombre_de_Municipio = municipios.Nombre_de_Municipio.apply(lambda i: i.strip())

# La columna Codigo_de_Provincia debe ser string, no se deberia poder operaciones matematicas sobre ella
municipios.Código_de_Provincia = [str(i) for i in municipios.Código_de_Provincia]

## Carga de la nueva tabla de elecciones - Solo Madrid
 

In [38]:
import pandas as pd
import os
import re

path = "resources/Tablas_elecciones_barrio/"
files = ""
tables = []

    

In [40]:
# Cargar los nombre de todos los archivos almacenados dentro de la carpeta "resources/Tablas_elecciones_barrio/"
for r, d, f in os.walk(path):
    files = f
# lectura y limpieza de las 21 tablas. Resultado: una lista de 21 tablas
for file in files:
    tmp_table = pd.read_excel(path + '{0}'.format(file), 
                              sep=";", encoding="ISO-8859-1")
    tmp_table.columns = [i.strip().replace(' ','_') for i in tmp_table.columns]
    tmp_table.reset_index(level=0, inplace=True)
    tmp_table.rename(columns={'index': 'Barrio'}, inplace=True)
    tmp_table['Distrito'] = re.sub("^[0-9]+(\.)(\s)", "", tmp_table.Barrio[0])
    tmp_table.drop([0,1], inplace=True) 
    tables.append(tmp_table)

# Se concatenan las 21 tablas en una sola
main_table_elecciones = pd.concat(tables)
# Se unifica el index para toda la tabla
main_table_elecciones.reset_index(drop=True, inplace=True)
main_table_elecciones
# main_table.tail()

# Pendientes
# 1 -La columna "Barrio" debe contener solo el nombre del barrio. Para el código se debe crear una nueva columna "Codigo"
# 2 -Crear una nueva columna en donde se puedan observar los porcentajes de abstención
#  OK 3- Poner el nombre del distrito (por ejemplo 'Centro') en una columna -"Distrito"- para todos los barrios que pertenecen a él


    

,Barrio,Censo_electoral,Abstención,Votos_nulos,Votos_blancos,Total_votos_a_candidaturas,PACMA,PH,P.S.O.E.,P.P.,...,IU-MpM,MCE,VOX,UPyD,M.I.,P.C.P.E.,CNTG+,PCTE,PCAS-TC-PPCCAL-PACTO,Distrito
0,011. Palacio,17.845,5.379,47.0,50.0,12.369,66.0,0.0,1.149,2.309,...,454.000,2.0,724.000,8.0,5.0,4.0,7.0,1.0,3.0,Centro
1,012. Embajadores,31.496,10.829,56.0,94.0,20.517,107.0,6.0,2.073,2.548,...,1.443,4.0,949.000,11.0,12.0,10.0,28.0,6.0,14.0,Centro
2,013. Cortes,7.755,2.466,16.0,27.0,5.246,24.0,2.0,449.000,1.071,...,188.000,1.0,300.000,3.0,1.0,2.0,6.0,3.0,1.0,Centro
3,014. Justicia,12.838,4.225,22.0,37.0,8.554,47.0,1.0,686.000,1.979,...,183.000,2.0,594.000,13.0,4.0,3.0,9.0,0.0,0.0,Centro
4,015. Universidad,23.738,7.683,48.0,56.0,15.951,78.0,3.0,1.428,2.784,...,527.000,2.0,822.000,11.0,4.0,6.0,19.0,5.0,6.0,Centro
5,016. Sol,5.201,1.923,6.0,9.0,3.263,13.0,1.0,284.000,688.000,...,92.000,0.0,236.000,1.0,1.0,2.0,6.0,2.0,1.0,Centro
6,021. Imperial,18.598,4.436,50.0,62.0,14.050,57.0,4.0,1.388,3.235,...,368.000,1.0,1.005,13.0,4.0,6.0,18.0,3.0,9.0,Arganzuela
7,022. Acacias,30.449,6.995,68.0,95.0,23.291,89.0,5.0,2.419,5.100,...,667.000,4.0,1.407,19.0,5.0,4.0,18.0,3.0,10.0,Arganzuela
8,023. Chopera,15.600,4.729,51.0,51.0,10.769,44.0,3.0,1.406,2.325,...,394.000,0.0,623.000,10.0,3.0,4.0,17.0,2.0,10.0,Arganzuela
9,024. Legazpi,14.431,3.252,24.0,57.0,11.098,36.0,2.0,1.053,1.860,...,249.000,2.0,739.000,13.0,10.0,4.0,12.0,5.0,5.0,Arganzuela


# Carga de la tabla del Padron por parrios 
Actulizada el 1 de Noviembre de 2019

In [43]:
import pandas as pd


# Cargar la tabla del Padron por barrios
padron = pd.read_excel("resources/padron_barrios.xls")


In [42]:
# Limpieza y organización de la tabla
tmp = []
separador_tabla = 0
for i in range(0, len(padron.Españoles_Hombres)):
    if pd.isna(padron.Españoles_Hombres[i]):
        dis_bar = padron.Edad[i].replace(" ", "").split("/")
        for j in  range(i+1, len(padron.Españoles_Hombres)):
            if pd.isna(padron.Españoles_Hombres[j]):
                tmp.append(padron.iloc[i:j,].copy(deep=True))
                tmp[separador_tabla].drop([i,i+1], inplace=True) 
                tmp[separador_tabla]['Distrito'] = dis_bar[0]
                tmp[separador_tabla]['Barrio'] = dis_bar[1]
                break
        separador_tabla += 1

main_table_padron = pd.concat(tmp)
main_table_padron.reset_index(drop=True, inplace=True)
main_table_padron
    
    

,Edad,Españoles_Hombres,Españoles_Mujeres,Españoles_Total,Extranjeros_Hombres,Extranjeros_Mujeres,Extranjeros_Total,Total_Hombres,Total_Mujeres,Total,Distrito,Barrio
0,0 - 4,263.0,238.0,501.0,56.0,50.0,106.0,319.0,288.0,607.0,CENTRO,PALACIO
1,5 - 9,235.0,265.0,500.0,43.0,38.0,81.0,278.0,303.0,581.0,CENTRO,PALACIO
2,10 - 14,227.0,238.0,465.0,30.0,38.0,68.0,257.0,276.0,533.0,CENTRO,PALACIO
3,15 - 19,234.0,282.0,516.0,44.0,55.0,99.0,278.0,337.0,615.0,CENTRO,PALACIO
4,20 - 24,323.0,346.0,669.0,161.0,212.0,373.0,484.0,558.0,1042.0,CENTRO,PALACIO
5,25 - 29,627.0,567.0,1194.0,319.0,434.0,753.0,946.0,1001.0,1947.0,CENTRO,PALACIO
6,30 - 34,833.0,729.0,1562.0,309.0,357.0,666.0,1142.0,1086.0,2228.0,CENTRO,PALACIO
7,35 - 39,850.0,759.0,1609.0,327.0,298.0,625.0,1177.0,1057.0,2234.0,CENTRO,PALACIO
8,40 - 44,1002.0,826.0,1828.0,236.0,171.0,407.0,1238.0,997.0,2235.0,CENTRO,PALACIO
9,45 - 49,931.0,772.0,1703.0,198.0,116.0,314.0,1129.0,888.0,2017.0,CENTRO,PALACIO


## En este punto, las tablas ya están listas para trabajar: main_table_padron, main_table_elecciones